In [38]:
import nltk
nltk.download('stopwords')
import re
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import logging
import warnings

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#can extend stopwords using the below command
#stop_words.extend(['from','subject','re','edu','use'])

In [6]:
df = pd.read_csv('C:\\Users\\antand\\Downloads\\posts-formatted-for-topicmodelling.csv', encoding='latin-1')

In [8]:
df.head()

,Unnamed: 0,year,username,text
0,1,2011,10171087,I brought Trevor to work!!! Haha #humanbone #t...
1,2,2011,10031678,My first #humanbone ! Thanx @USERNAME_HERE #go...
2,3,2011,10171087,My new torso came in today! I love it!! #human...
3,4,2011,10171087,Just got 2 femurs and a hand!!!! Yaaaay! #huma...
4,5,2011,10321446,Dead hooker mistle toe...


In [23]:
#tokenize words and clean-up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True)) #deacc = True removes punctuation

data_words = list(sent_to_words(df.text))
#print(data_words)

In [52]:
#Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words,min_count=5,threshold=100) #higher threshold fewer phrases
trigram = gensim.models.Phrases(bigram[data_words],threshold=10)

In [53]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [54]:
#this will combine trigrams into 3 words with underscores so "pls fix ASAP" turns into "pls_fix_ASAP" if those 3 words occur with enough frequency together
print(trigram_mod[bigram_mod[data_words]])

In [55]:
#function to remove stop words
def remove_stopwords(texts):
    return[[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

#function to make bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

#function to make trigrams
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [56]:
data_words_nostops = remove_stopwords(data_words)

In [57]:
data_words_trigrams = make_trigrams(data_words_nostops)

In [58]:
print(data_words_trigrams)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [59]:
#create Dictionary
id2word = corpora.Dictionary(data_words_trigrams)

#create Corpus
texts = data_words_trigrams

#term document frequency
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(1, 1), (3, 1), (7, 1), (8, 1), (9, 1)], [(1, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2)], [(1, 1), (3, 1), (12, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(23, 1), (24, 1), (25, 1), (26, 1)], [(27, 1), (28, 1), (29, 1), (30, 1), (31, 1)], [(1, 1), (3, 1), (12, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)], [(15, 1), (16, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(11, 1), (14, 1), (43, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1)], [(3, 1), (53, 1), (54, 1), (55, 1)], [(3, 1), (56, 1), (57, 1), (58, 1)], [(3, 1), (59, 1), (60, 1)], [(3, 1), (61, 1), (62, 1)], [(63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)], [(69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1)], [(9, 1), (15, 1), (80, 1)], [(81, 1)], [(47, 1), (82, 1), (83, 1), (84, 1)], [(3, 1), (85, 1), (86, 1), (87,

In [60]:
#human readable format of corpus (term-frequency)
[[(id2word[id],freq) for id, freq in cp] for cp in corpus]

[[('brought', 1),
  ('goldenlotus', 1),
  ('haha', 1),
  ('humanbone', 1),
  ('tattoos', 1),
  ('trevor', 1),
  ('work', 1)],
 [('goldenlotus', 1),
  ('humanbone', 1),
  ('first', 1),
  ('thanx', 1),
  ('username_here', 1)],
 [('goldenlotus', 1),
  ('came', 1),
  ('human', 1),
  ('humanremains', 1),
  ('love', 1),
  ('new', 1),
  ('realbone', 1),
  ('skeleton', 1),
  ('today', 1),
  ('torso', 2)],
 [('goldenlotus', 1),
  ('humanbone', 1),
  ('humanremains', 1),
  ('femurs', 1),
  ('got', 1),
  ('hand', 1),
  ('yaaaay', 1)],
 [('dead', 1), ('hooker', 1), ('mistle', 1), ('toe', 1)],
 [('bought', 1), ('horn', 1), ('human_femur', 1), ('mas', 1), ('tibet', 1)],
 [('goldenlotus', 1),
  ('humanbone', 1),
  ('humanremains', 1),
  ('art', 1),
  ('boneart', 1),
  ('complete', 1),
  ('door', 1),
  ('feel_like', 1),
  ('front', 1),
  ('hope', 1),
  ('like', 1),
  ('named', 1),
  ('satan', 1),
  ('working', 1)],
 [('realbone', 1),
  ('skeleton', 1),
  ('bone', 1),
  ('cat', 1),
  ('lostplace', 1),


In [61]:
#Build LDA model
#chunksize = number of docuemtns to be used in each training chunk
#update_everey = how often the model parameters should be updated
#passes = total number of training passes
lda = gensim.models.ldamodel.LdaModel(corpus, num_topics=25)

In [62]:
pprint(lda.print_topics())
doc_lda = lda[corpus]

[(2,
  '0.032*"340" + 0.021*"47" + 0.015*"4181" + 0.014*"7755" + 0.010*"1189" + '
  '0.010*"1591" + 0.010*"1303" + 0.010*"562" + 0.009*"283" + 0.008*"8196"'),
 (6,
  '0.022*"1841" + 0.021*"2049" + 0.020*"1727" + 0.017*"1754" + 0.014*"5318" + '
  '0.012*"7113" + 0.011*"9" + 0.011*"2051" + 0.011*"2894" + 0.011*"7493"'),
 (15,
  '0.103*"47" + 0.097*"340" + 0.024*"105" + 0.022*"210" + 0.021*"32" + '
  '0.020*"1718" + 0.019*"147" + 0.018*"1513" + 0.017*"1486" + 0.016*"72"'),
 (4,
  '0.091*"47" + 0.083*"340" + 0.022*"147" + 0.018*"72" + 0.015*"9" + '
  '0.015*"11" + 0.014*"32" + 0.012*"607" + 0.011*"16" + 0.010*"1144"'),
 (8,
  '0.035*"2695" + 0.030*"2086" + 0.025*"4073" + 0.021*"7901" + 0.015*"8825" + '
  '0.013*"7359" + 0.012*"3026" + 0.011*"340" + 0.010*"1081" + 0.009*"2787"'),
 (11,
  '0.019*"4631" + 0.017*"47" + 0.015*"511" + 0.014*"340" + 0.012*"2147" + '
  '0.010*"2638" + 0.009*"1855" + 0.009*"4628" + 0.008*"2636" + 0.008*"5547"'),
 (13,
  '0.025*"716" + 0.024*"6561" + 0.024*"2734" + 

In [63]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      19.607990        1       1  0.266487 -0.041405
20      9.920568        1       2  0.224769 -0.102553
15      8.585763        1       3  0.246397  0.073519
0       7.333598        1       4  0.129177 -0.161617
21      6.883753        1       5  0.167047  0.031796
1       6.560638        1       6  0.186780  0.027981
9       5.581255        1       7  0.164200  0.170034
13      3.123753        1       8 -0.033845 -0.125562
16      2.954792        1       9  0.058041  0.109827
14      2.512916        1      10  0.006546  0.025469
2       2.368412        1      11  0.001319 -0.066938
10      2.339248        1      12 -0.047309  0.017737
5       2.287060        1      13 -0.070800 -0.037218
17      2.223272        1      14  0.027885  0.027586
23      2.171383        1      15 -0.108858  0.019274
12      2.103791        1      16 -0.066655 -0.062403
24      1.928901        1      17 -0.108151 -0.001432
19      1.914036        1      18 -0.095847 -0.031375
22      1.601291        1      19 -0.082119 -0.026095
3       1.516790        1      20 -0.167231  0.011972
7       1.348655        1      21 -0.131970  0.063718
6       1.322761        1      22 -0.153776  0.040631
11      1.298607        1      23 -0.107327 -0.000606
8       1.284346        1      24 -0.135088  0.021387
18      1.226421        1      25 -0.169672  0.016274, topic_info=     Category          Freq                         Term         Total  \
term                                                                     
47    Default  16071.000000                        skull  16071.000000   
32    Default   2439.000000                          art   2439.000000   
4380  Default    865.000000                  macabre_art    865.000000   
340   Default  14081.000000                   humanskull  14081.000000   
548   Default   1300.000000                      drawing   1300.000000   
1144  Default   1730.000000                       tattoo   1730.000000   
2714  Default    885.000000                  skulltattoo    885.000000   
9     Default   2530.000000                username_here   2530.000000   
4064  Default    630.000000                 illustration    630.000000   
2051  Default    555.000000                     painting    555.000000   
779   Default    633.000000                blackandwhite    633.000000   
787   Default    571.000000                       artist    571.000000   
1513  Default    773.000000                       sketch    773.000000   
716   Default    632.000000                          com    632.000000   
147   Default   3547.000000                       skulls   3547.000000   
6561  Default    336.000000                 blackandgrey    336.000000   
1718  Default    693.000000                          ink    693.000000   
8492  Default    240.000000                   sketchbook    240.000000   
105   Default    628.000000  username_here_username_here    628.000000   
1405  Default   1129.000000                    halloween   1129.000000   
11    Default   2475.000000                        human   2475.000000   
2930  Default    253.000000                      pendant    253.000000   
621   Default    514.000000                      artwork    514.000000   
1553  Default    379.000000                       carved    379.000000   
7850  Default    252.000000                     inktober    252.000000   
1969  Default    324.000000                          wip    324.000000   
6026  Default    272.000000                     instaart    272.000000   
5528  Default    405.000000                      darkart    405.000000   
1217  Default    299.000000                           de    299.000000   
210   Default   1003.000000                        death   1003.000000   
...       ...           ...                          ...           ...   
4363  Topic25     41.089153                      session 

In [64]:
#calculate model perplexity
#a measure of how good the model is, low the score the better
print('\nPerplexity:',lda.log_perplexity(corpus))


Perplexity: -16.458212618148018


In [51]:
#calculate model coherence score
coherence_model_lda = CoherenceModel(model=lda,texts=data_words_trigrams,dictionary=id2word,coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score:',coherence_lda)


Coherence Score: 0.4330338370163891
